In [23]:
explicador_codigo_tool = {
    'type': 'function',
    'function': {
        'name': 'explicador_codigo',
        'description': 'Explica paso a paso el funcionamiento del código fuente dado, línea por línea o por bloques',
        'parameters': {
            'type': 'object',
            'properties': {
                'codigo': {
                    'type': 'string',
                    'description': 'El código fuente a explicar',
                }
            },
            'required': ['lenguaje', 'codigo'],
        },
    },
}
def explicador_codigo(codigo=""):
    explicacion = []
    lineas = codigo.strip().split('\n')
    for i, linea in enumerate(lineas, 1):
        if not linea.strip():
            continue
        explicacion.append(f"Línea {i}: `{linea.strip()}` — Esta línea...")
    return "\n".join(explicacion)

In [24]:
code = """
x = 1
y = 2
r = x+y
"""
explicador_codigo(code)

'Línea 1: `x = 1` — Esta línea...\nLínea 2: `y = 2` — Esta línea...\nLínea 3: `r = x+y` — Esta línea...'

In [25]:
texto_a_tabla_sqlite_tool = {
    'type': 'function',
    'function': {
        'name': 'texto_a_tabla_sqlite',
        'description': 'Convierte texto libre en una tabla SQLite y ejecuta los comandos en una base de datos local',
        'parameters': {
            'type': 'object',
            'properties': {
                'texto': {
                    'type': 'string',
                    'description': 'Texto con registros o listas que deben convertirse a SQL'
                },
                'nombre_tabla': {
                    'type': 'string',
                    'description': 'Nombre deseado para la tabla en la base de datos SQLite'
                }
            },
            'required': ['texto', 'nombre_tabla']
        }
    }
}

import sqlite3
import os
import re
import ollama

def texto_a_tabla_sqlite(texto="", nombre_tabla="tabla"):
    db_path = "data.db"

    # Crear archivo .db si no existe
    if not os.path.exists(db_path):
        open(db_path, 'w').close()

    # Prompt para Ollama: solo SQL, sin explicaciones ni markdown
    prompt = f"""
Convierte el siguiente texto en código SQL válido para SQLite.

Nombre de la tabla: {nombre_tabla}

Texto:
{texto}

Devuelve solamente SQL, sin explicaciones, comentarios, encabezados, ni etiquetas como ```sql. El resultado debe empezar con CREATE TABLE y terminar con los INSERTs.
    """

    # Llamada a Ollama
    response = ollama.chat(
        model="llama3.2:latest",
        messages=[{"role": "user", "content": prompt}]
    )

    # Extraer solo el SQL desde "CREATE TABLE"
    sql_code_raw = response["message"]["content"]
    match = re.search(r'CREATE TABLE[\s\S]+', sql_code_raw, re.IGNORECASE)

    if not match:
        return f"❌ No se encontró una sentencia CREATE TABLE válida.\n\n📄 Contenido:\n{sql_code_raw}"

    sql_code_dirty = match.group(0)

    # Eliminar líneas no ejecutables (markdown, notas)
    sql_code_lines = sql_code_dirty.splitlines()
    sql_code = "\n".join(
        line for line in sql_code_lines
        if not line.strip().startswith("```") and "nota:" not in line.lower()
    )

    # Ejecutar SQL
    try:
        conn = sqlite3.connect(db_path)
        cursor = conn.cursor()
        cursor.executescript(sql_code)
        conn.commit()

        # Verificar registros
        cursor.execute(f"SELECT * FROM {nombre_tabla} LIMIT 5;")
        rows = cursor.fetchall()
        columns = [desc[0] for desc in cursor.description]
        conn.close()

        # Resultado
        preview = "\n".join(str(dict(zip(columns, row))) for row in rows)
        return f"Base de datos `data.db` creada y tabla `{nombre_tabla}` insertada con éxito.\n\n📋 Registros de ejemplo:\n{preview}"

    except Exception as e:
        return sql_code

In [26]:
crear_carpeta_tool = {
    'type': 'function',
    'function': {
        'name': 'crear_carpeta',
        'description': 'Crea una carpeta (directorio) en el sistema de archivos',
        'parameters': {
            'type': 'object',
            'properties': {
                'ruta': {
                    'type': 'string',
                    'description': 'Ruta absoluta o relativa donde se debe crear la carpeta. Por ejemplo: "docs/nuevo_proyecto"'
                },
                'crear_subcarpetas': {
                    'type': 'boolean',
                    'description': 'Si es verdadero, crea todas las carpetas intermedias si no existen',
                    'default': True
                }
            },
            'required': ['ruta']
        }
    }
}
import os

def crear_carpeta(ruta, crear_subcarpetas=True):
    try:
        if crear_subcarpetas:
            os.makedirs(ruta, exist_ok=True)
        else:
            os.mkdir(ruta)  # Falla si la ruta intermedia no existe

        return f"✅ Carpeta creada correctamente en: {ruta}"
    except FileExistsError:
        return f"⚠️ La carpeta ya existe: {ruta}"
    except Exception as e:
        return f"❌ Error al crear la carpeta: {str(e)}"

In [ ]:
import ollama
tool_history = []
def tool_calling(query):
    messages = [{'role': 'user', 'content': query}]

    response = ollama.chat(
        model='llama3.2:latest',
        messages=messages,
        tools=[explicador_codigo, texto_a_tabla_sqlite,crear_carpeta],
    )
    print(response['message'])
    messages.append(response['message'])

    if not response['message'].get('tool_calls'):
        print("El modelo no utilizó la función. Su respuesta fue:")
        print(response['message']['content'])

    if response['message'].get('tool_calls'):
        available_functions = {
            'explicador_codigo' : explicador_codigo,
            'texto_a_tabla_sqlite' : texto_a_tabla_sqlite,
            'crear_carpeta' : crear_carpeta
        }
    for tool in response['message']['tool_calls']:

        function_to_call = available_functions[tool['function']['name']]
        args = tool['function']['arguments'].values()
        function_response = function_to_call(*args)

        print("\nTool Response:\n", function_response)

        messages.append({'role': 'tool', 'content': function_response})

    final_response = ollama.chat(model='llama3.2:latest', messages=messages)

    return final_response['message']['content']

In [28]:
query = """
x = 1
y = 2
r = x+y
"""
result = tool_calling(query)
print("\n\nRespuesta:\n", result)

role='assistant' content='' thinking=None images=None tool_calls=[ToolCall(function=Function(name='explicador_codigo', arguments={'codigo': 'print('}))]

Tool Response:
 Línea 1: `codigo` — Esta línea...


Respuesta:
 No parece ser un código válido en Python. Es posible que sea una instrucción de ayuda o una marca para destacar el código.

En cualquier caso, si queremos ejecutar ese código, podemos convertirlo a un programa Python válido:

```
print("x = 1\ny = 2\nr = x+y")
```

Al ejecutar este código en un entorno de Python (como Jupyter Notebook o un editor de texto con soporte para Python), obtendremos la siguiente salida:

```
x = 1
y = 2
r = x + y
```


In [29]:
query = """
Tengo esta lista de productos:
- Café especial, 12000
- Té verde, 8000
- Chocolate oscuro, 9500

Crea una tabla SQLite llamada 'productos'
"""
tool_calling(query)

role='assistant' content='' thinking=None images=None tool_calls=[ToolCall(function=Function(name='texto_a_tabla_sqlite', arguments={'nombre_tabla': 'productos', 'texto': 'Tengo esta lista de productos:\n- Café especial, 12000\n- Té verde, 8000\n- Chocolate oscuro, 9500'}))]

Tool Response:
 Base de datos `data.db` creada y tabla `texto` insertada con éxito.

📋 Registros de ejemplo:
{'nombre_tabla': 'texto'}
{'nombre_tabla': 'otroTexto'}


'Aquí tienes el código SQLite para crear una tabla llamada "productos":\n```sql\nCREATE TABLE productos (\n    id INTEGER PRIMARY KEY,\n    nombre TEXT NOT NULL,\n    precio REAL NOT NULL\n);\n```\nEstablece que cada registro en la tabla tenga un identificador único, un nombre de producto y un precio.\n\nAgrega los productos a la tabla:\n```sql\nINSERT INTO productos (nombre, precio) VALUES \n(\'Café especial\', 12000),\n(\'Té verde\', 8000),\n(\'Chocolate oscuro\', 9500);\n```\nEsto insertará los productos en la tabla con sus respectivos precios.\n\nMuestra los registros de la tabla:\n```sql\nSELECT * FROM productos;\n```\nEsto mostrará todos los registros de la tabla.\n\nRecuerda que debes ejecutar el comando `CREATE TABLE` para crear la tabla, y luego los comandos `INSERT INTO` y `SELECT * FROM` para agregar y mostrar datos.'

In [30]:
query = 'Crea una carpeta llamada documentos/'
tool_calling(query)

role='assistant' content='' thinking=None images=None tool_calls=[ToolCall(function=Function(name='crear_carpeta', arguments={'crear_subcarpetas': '', 'ruta': 'documentos/'}))]

Tool Response:
 ✅ Carpeta creada correctamente en: crear_subcarpetas


'¡Excelente! La carpeta "documentos" ha sido creada con éxito. ¿Necesitas algo más?'